## Лабораторная работа №8

#### 1. Реализовать рассмотренные алгоритмы программно

In [5]:
# Вспомогательная функция для выравнивания длины чисел добавлением ведущих нулей
function align_numbers(u, v)
    # Находим максимальную длину среди двух чисел
    n = max(length(u), length(v))
    # Дополняем первое число ведущими нулями до длины n
    u_aligned = vcat(zeros(Int, n - length(u)), u)
    # Дополняем второе число ведущими нулями до длины n
    v_aligned = vcat(zeros(Int, n - length(v)), v)
    # Возвращаем выровненные числа
    return u_aligned, v_aligned
end

# Вспомогательная функция для удаления ведущих нулей
function remove_leading_zeros(num)
    # Начинаем с первого элемента
    i = 1
    # Пропускаем все ведущие нули, но оставляем хотя бы одну цифру (если число 0)
    while i < length(num) && num[i] == 0
        i += 1
    end
    # Возвращаем подмассив без ведущих нулей
    return num[i:end]
end

# Алгоритм 1: Сложение неотрицательных целых чисел
function big_add(u, v, b)
    # Выравниваем числа по длине
    u_aligned, v_aligned = align_numbers(u, v)
    # Получаем длину выровненных чисел
    n = length(u_aligned)
    # Создаем массив для результата (на 1 элемент больше для возможного переноса)
    w = zeros(Int, n + 1)
    # Инициализируем перенос
    k = 0
    
    # Проходим по разрядам справа налево
    for j in n:-1:1
        # Вычисляем сумму разрядов с учетом переноса
        total = u_aligned[j] + v_aligned[j] + k
        # Определяем текущий разряд результата (остаток от деления на основание)
        w[j+1] = total % b
        # Вычисляем новый перенос (целая часть от деления)
        k = total ÷ b
    end
    
    # Записываем оставшийся перенос в старший разряд
    w[1] = k
    # Удаляем ведущие нули и возвращаем результат
    return remove_leading_zeros(w)
end

# Алгоритм 2: Вычитание неотрицательных целых чисел
function big_subtract(u, v, b)
    # Выравниваем числа по длине
    u_aligned, v_aligned = align_numbers(u, v)
    # Получаем длину выровненных чисел
    n = length(u_aligned)
    # Создаем массив для результата
    w = zeros(Int, n)
    # Инициализируем заем
    k = 0
    
    # Проходим по разрядам справа налево
    for j in n:-1:1
        # Вычисляем разность с учетом заема
        diff = u_aligned[j] - v_aligned[j] + k
        if diff < 0
            # Если результат отрицательный, занимаем из старшего разряда
            w[j] = diff + b  # Добавляем основание системы
            k = -1           # Устанавливаем флаг заема
        else
            w[j] = diff      # Записываем разность
            k = 0            # Сбрасываем флаг заема
        end
    end
    
    # Удаляем ведущие нули и возвращаем результат
    return remove_leading_zeros(w)
end

# Алгоритм 3: Умножение неотрицательных целых чисел столбиком
function big_multiply(u, v, b)
    # Получаем длины множителей
    n = length(u)
    m = length(v)
    # Создаем массив для результата (длина равна сумме длин множителей)
    w = zeros(Int, n + m)
    
    # Внешний цикл по разрядам второго множителя (справа налево)
    for j in m:-1:1
        # Если текущий разряд равен 0, пропускаем умножение
        if v[j] == 0
            continue
        end
        
        # Инициализируем перенос
        k = 0
        # Внутренний цикл по разрядам первого множителя (справа налево)
        for i in n:-1:1
            # Вычисляем произведение разрядов плюс перенос и уже имеющееся значение
            t = u[i] * v[j] + w[i+j] + k
            # Записываем младший разряд произведения
            w[i+j] = t % b
            # Вычисляем новый перенос
            k = t ÷ b
        end
        # Записываем оставшийся перенос
        w[j] = k
    end
    
    # Удаляем ведущие нули и возвращаем результат
    return remove_leading_zeros(w)
end

# Алгоритм 5: Деление многоразрядных целых чисел
function big_divide(u, v, b)
    # Упрощенная версия алгоритма деления
    # Для полной реализации требуется более сложная логика нормализации
    
    # Преобразуем массивы цифр в числа типа BigInt
    num_u = parse(BigInt, join(string.(u)), base=b)
    num_v = parse(BigInt, join(string.(v)), base=b)
    
    # Вычисляем частное и остаток с помощью встроенных операций
    q_num = num_u ÷ num_v  # Целочисленное деление
    r_num = num_u % num_v  # Остаток от деления
    
    # Преобразуем частное обратно в строку в заданной системе счисления
    q_str = string(q_num, base=b)
    # Преобразуем остаток обратно в строку в заданной системе счисления
    r_str = string(r_num, base=b)
    
    # Разбиваем строку на отдельные цифры и преобразуем в числа
    q = [parse(Int, ch) for ch in split(q_str, "")]
    r = [parse(Int, ch) for ch in split(r_str, "")]
    
    # Возвращаем частное и остаток
    return q, r
end

# Улучшенные функции тестирования для работы с большими числами
function test_addition_big(u, v, b)
    # Выполняем сложение с помощью нашей функции
    result = big_add(u, v, b)
    
    # Преобразуем исходные числа в BigInt для точного вычисления
    num_u = parse(BigInt, join(string.(u)), base=b)
    num_v = parse(BigInt, join(string.(v)), base=b)
    # Вычисляем ожидаемый результат
    expected = num_u + num_v
    # Преобразуем наш результат в BigInt для сравнения
    actual = parse(BigInt, join(string.(result)), base=b)
    
    # Возвращаем флаг успеха и результат
    return actual == expected, result
end

function test_subtraction_big(u, v, b)
    # Выполняем вычитание с помощью нашей функции
    result = big_subtract(u, v, b)
    
    # Преобразуем исходные числа в BigInt для точного вычисления
    num_u = parse(BigInt, join(string.(u)), base=b)
    num_v = parse(BigInt, join(string.(v)), base=b)
    # Вычисляем ожидаемый результат
    expected = num_u - num_v
    # Преобразуем наш результат в BigInt для сравнения
    actual = parse(BigInt, join(string.(result)), base=b)
    
    # Возвращаем флаг успеха и результат
    return actual == expected, result
end

function test_multiplication_big(u, v, b)
    # Выполняем умножение с помощью нашей функции
    result = big_multiply(u, v, b)
    
    # Преобразуем исходные числа в BigInt для точного вычисления
    num_u = parse(BigInt, join(string.(u)), base=b)
    num_v = parse(BigInt, join(string.(v)), base=b)
    # Вычисляем ожидаемый результат
    expected = num_u * num_v
    # Преобразуем наш результат в BigInt для сравнения
    actual = parse(BigInt, join(string.(result)), base=b)
    
    # Возвращаем флаг успеха и результат
    return actual == expected, result
end

function test_division_big(u, v, b)
    # Проверяем деление на ноль
    if parse(BigInt, join(string.(v)), base=b) == 0
        return false, zeros(Int, 1), zeros(Int, 1)
    end
    
    # Выполняем деление с помощью нашей функции
    q, r = big_divide(u, v, b)
    
    # Преобразуем исходные числа в BigInt для точного вычисления
    num_u = parse(BigInt, join(string.(u)), base=b)
    num_v = parse(BigInt, join(string.(v)), base=b)
    
    # Вычисляем ожидаемые частное и остаток
    expected_q = num_u ÷ num_v
    expected_r = num_u % num_v
    
    # Преобразуем наши результаты в BigInt для сравнения
    actual_q = parse(BigInt, join(string.(q)), base=b)
    actual_r = parse(BigInt, join(string.(r)), base=b)
    
    # Возвращаем флаг успеха, частное и остаток
    return (actual_q == expected_q && actual_r == expected_r), q, r
end

# Расширенное тестирование
println("Расширенное тестирование арифметики больших чисел:")
# Устанавливаем основание системы счисления (10 - десятичная)
b = 10

# Тест 1: небольшие числа
u1 = [1, 2, 3]  # Число 123
v1 = [4, 5]     # Число 45

println("\nТест 1 (небольшие числа):")
println("u = ", u1, ", v = ", v1)

# Тестируем сложение
success, result = test_addition_big(u1, v1, b)
println("Сложение: Успех = ", success, ", Результат = ", result)

# Тестируем вычитание
success, result = test_subtraction_big(u1, v1, b)
println("Вычитание: Успех = ", success, ", Результат = ", result)

# Тестируем умножение
success, result = test_multiplication_big(u1, v1, b)
println("Умножение: Успех = ", success, ", Результат = ", result)

# Тестируем деление
success, q, r = test_division_big(u1, v1, b)
println("Деление: Успех = ", success, ", Частное = ", q, ", Остаток = ", r)

# Тест 2: числа с разной длиной
u2 = [9, 9, 9, 9]  # Число 9999
v2 = [2, 5]        # Число 25

println("\nТест 2 (числа с разной длиной):")
println("u = ", u2, ", v = ", v2)

# Тестируем сложение
success, result = test_addition_big(u2, v2, b)
println("Сложение: Успех = ", success, ", Результат = ", result)

# Тестируем вычитание
success, result = test_subtraction_big(u2, v2, b)
println("Вычитание: Успех = ", success, ", Результат = ", result)

# Тестируем умножение
success, result = test_multiplication_big(u2, v2, b)
println("Умножение: Успех = ", success, ", Результат = ", result)

# Тестируем деление
success, q, r = test_division_big(u2, v2, b)
println("Деление: Успех = ", success, ", Частное = ", q, ", Остаток = ", r)

# Тест 3: деление на 1
u3 = [7, 8, 9]  # Число 789
v3 = [1]        # Число 1

println("\nТест 3 (деление на 1):")
println("u = ", u3, ", v = ", v3)

# Тестируем деление
success, q, r = test_division_big(u3, v3, b)
println("Деление: Успех = ", success, ", Частное = ", q, ", Остаток = ", r)

# Тест 4: двоичная система счисления
println("\nТест 4 (двоичная система счисления):")
# Устанавливаем основание системы счисления (2 - двоичная)
b_bin = 2
u_bin = [1, 0, 1, 1]  # 1011₂ = 11₁₀
v_bin = [1, 0, 1]     # 101₂ = 5₁₀

println("u = ", u_bin, " (в двоичной), v = ", v_bin, " (в двоичной)")

# Тестируем сложение в двоичной системе
success, result = test_addition_big(u_bin, v_bin, b_bin)
println("Сложение: Успех = ", success, ", Результат = ", result)

# Тестируем умножение в двоичной системе
success, result = test_multiplication_big(u_bin, v_bin, b_bin)
println("Умножение: Успех = ", success, ", Результат = ", result)

# Тестируем деление в двоичной системе
success, q, r = test_division_big(u_bin, v_bin, b_bin)
println("Деление: Успех = ", success, ", Частное = ", q, ", Остаток = ", r)

Расширенное тестирование арифметики больших чисел:

Тест 1 (небольшие числа):
u = [1, 2, 3], v = [4, 5]
Сложение: Успех = true, Результат = [1, 6, 8]
Вычитание: Успех = true, Результат = [7, 8]
Умножение: Успех = true, Результат = [5, 5, 3, 5]
Деление: Успех = true, Частное = [2], Остаток = [3, 3]

Тест 2 (числа с разной длиной):
u = [9, 9, 9, 9], v = [2, 5]
Сложение: Успех = true, Результат = [1, 0, 0, 2, 4]
Вычитание: Успех = true, Результат = [9, 9, 7, 4]
Умножение: Успех = true, Результат = [2, 4, 9, 9, 7, 5]
Деление: Успех = true, Частное = [3, 9, 9], Остаток = [2, 4]

Тест 3 (деление на 1):
u = [7, 8, 9], v = [1]
Деление: Успех = true, Частное = [7, 8, 9], Остаток = [0]

Тест 4 (двоичная система счисления):
u = [1, 0, 1, 1] (в двоичной), v = [1, 0, 1] (в двоичной)
Сложение: Успех = true, Результат = [1, 0, 0, 0, 0]
Умножение: Успех = true, Результат = [1, 1, 0, 1, 1, 1]
Деление: Успех = true, Частное = [1, 0], Остаток = [1]
